In [22]:
# Import Library Python
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
import os
import nltk

# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Job Posting

In [23]:
# Load and preprocess dataset
postings = pd.read_csv('postings.csv')
postings

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.715870e+12,NaN,NaN,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.715488e+12,NaN,NaN,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.716044e+12,NaN,NaN,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,3906267117,Lozano Smith,Title IX/Investigations Attorney,Our Walnut Creek office is currently seeking a...,195000.0,YEARLY,"Walnut Creek, CA",56120.0,1.0,NaN,...,1.716163e+12,NaN,Mid-Senior level,NaN,1.713571e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
123845,3906267126,Pinterest,"Staff Software Engineer, ML Serving Platform",About Pinterest:\n\nMillions of people across ...,NaN,NaN,United States,1124131.0,3.0,NaN,...,1.716164e+12,NaN,Mid-Senior level,NaN,1.713572e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN
123846,3906267131,EPS Learning,"Account Executive, Oregon/Washington",Company Overview\n\nEPS Learning is a leading ...,NaN,NaN,"Spokane, WA",90552133.0,3.0,NaN,...,1.716164e+12,NaN,Mid-Senior level,NaN,1.713572e+12,epsoperations.bamboohr.com,0,FULL_TIME,NaN,NaN
123847,3906267195,Trelleborg Applied Technologies,Business Development Manager,The Business Development Manager is a 'hunter'...,NaN,NaN,"Texas, United States",2793699.0,4.0,NaN,...,1.716165e+12,NaN,NaN,NaN,1.713573e+12,NaN,0,FULL_TIME,NaN,NaN


## Preprocessing Data

In [24]:
postings = postings.dropna(subset=['description'])
postings.isnull().sum()

job_id                             0
company_name                    1718
title                              0
description                        0
max_salary                     94050
pay_period                     87770
location                           0
company_id                      1716
views                           1689
med_salary                    117562
min_salary                     94050
formatted_work_type                0
applies                       100524
original_listed_time               0
remote_allowed                108599
job_posting_url                    0
application_url                36658
application_type                   0
expiry                             0
closed_time                   122771
formatted_experience_level     29402
skills_desc                   121403
listed_time                        0
posting_domain                 39961
sponsored                          0
work_type                          0
currency                       87770
c

In [25]:
# Preprocess text function
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase
        text = text.lower()
        # Remove URLs and special characters
        text = re.sub(r'@\S+', '', text)
        text = re.sub(r'https://\S+', '', text)
        text = re.sub(r'$\S+', '', text)
        # Remove symbols
        text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        # Remove stopwords
        stop = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word not in stop])
        return text
    return ''

# Apply preprocessing
clean_postings = postings.copy()
clean_postings['company_name'] = clean_postings['company_name'].apply(preprocess_text)
clean_postings['title'] = clean_postings['title'].apply(preprocess_text)
clean_postings['description'] = clean_postings['description'].apply(preprocess_text)
clean_postings['skills_desc'] = clean_postings['skills_desc'].apply(preprocess_text)

pattern = r'[^A-Za-z\s]'
postings['work_type'] = postings.work_type.str.replace(pattern, ' ', regex=True)

clean_postings['combined'] = clean_postings['company_name'] + ' ' + clean_postings['title'] + ' ' + clean_postings['description'] + ' ' + clean_postings['skills_desc']
clean_postings

C:\Users\HP\AppData\Local\Temp\ipykernel_15816\217509214.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postings['work_type'] = postings.work_type.str.replace(pattern, ' ', regex=True)


,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,combined
0,921716,corcoran sawyer smith,marketing coordinator,job descriptiona leading real estate firm new ...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,NaN,NaN,requirements seeking college graduate student ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,corcoran sawyer smith marketing coordinator jo...
1,1829192,,mental health therapistcounselor,aspen therapy wellness committed serving clien...,50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,NaN,,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,mental health therapistcounselor aspen therap...
2,10998357,national exemplar,assitant restaurant manager,national exemplar accepting applications assis...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,NaN,NaN,currently accepting resumes foh asisstant rest...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,national exemplar assitant restaurant manager ...
3,23221523,abrams fensterman llp,senior elder law trusts estates associate atto...,senior associate attorney elder law trusts est...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,NaN,NaN,position requires baseline understanding onlin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,abrams fensterman llp senior elder law trusts ...
4,35982263,,service technician,looking hvac service tech experience commerica...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,NaN,,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,service technician looking hvac service tech ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123844,3906267117,lozano smith,title ixinvestigations attorney,walnut creek office currently seeking attorney...,195000.0,YEARLY,"Walnut Creek, CA",56120.0,1.0,NaN,...,NaN,Mid-Senior level,,1.713571e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,lozano smith title ixinvestigations attorney w...
123845,3906267126,pinterest,staff software engineer ml serving platform,pinterest millions people across world come pi...,NaN,NaN,United States,1124131.0,3.0,NaN,...,NaN,Mid-Senior level,,1.713572e+12,www.pinterestcareers.com,0,FULL_TIME,NaN,NaN,pinterest staff software engineer ml serving p...
123846,3906267131,eps learning,account executive oregonwashington,company overview eps learning leading k12 supp...,NaN,NaN,"Spokane, WA",90552133.0,3.0,NaN,...,NaN,Mid-Senior level,,1.713572e+12,epsoperations.bamboohr.com,0,FULL_TIME,NaN,NaN,eps learning account executive oregonwashingto...
123847,3906267195,trelleborg applied technologies,business development manager,business development manager hunter carries ta...,NaN,NaN,"Texas, United States",2793699.0,4.0,NaN,...,NaN,NaN,,1.713573e+12,NaN,0,FULL_TIME,NaN,NaN,trelleborg applied technologies business devel...


In [26]:
clean_postings = clean_postings[['job_id', 'company_name', 'title', 'description', 'skills_desc', 'combined']]
clean_postings.to_csv('clean_postings.csv', index=False, sep=',')

# Data Profil

## Preprocessing

In [11]:
# Load and preprocess dataset
profiles = pd.read_csv('profiles.csv')
profiles

,timestamp,id,name,city,country_code,region,current_company:company_id,current_company:name,position,following,...,people_also_viewed,educations_details,education,avatar,languages,certifications,recommendations,recommendations_count,volunteer_experience,сourses
0,2023-01-10,catherinemcilkenny,"Catherine Fitzpatrick (McIlkenny), B.A",Canada,CA,NaN,NaN,NaN,Snr Business Analyst at Emploi et Développemen...,NaN,...,"[{""profile_link"":""https://ca.linkedin.com/in/l...",Queen's University Belfast,"[{""degree"":""Bachelor of Arts (B.A.) Honours"",""...",https://media.licdn.com/dms/image/C4E03AQEcz_j...,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-17,margot-bon-51a04624,Margot Bon,"The Randstad, Netherlands",NL,EU,gemeente-utrecht,Gemeente Utrecht,Communicatieadviseur Corporate & Strategie Gem...,NaN,...,"[{""profile_link"":""https://nl.linkedin.com/in/j...",NaN,"[{""degree"":""Scrum en Agile werken"",""end_year"":...",https://static.licdn.com/sc/h/244xhbkr7g40x6bs...,"[{""subtitle"":""Professional working proficiency...","[{""meta"":""Issued Jun 2013"",""subtitle"":""Van der...",Menno H. Poort “Ik werk al jaren prettig met M...,2.0,"[{""cause"":"""",""duration"":""Sep 2010 Jul 2020 9 y...",NaN
2,2023-05-17,mike-dean-8509a193,Mike Dean,"England, United Kingdom",UK,NaN,network-rail,Network Rail,Network Data Manager at Network Rail,NaN,...,"[{""profile_link"":""https://uk.linkedin.com/in/g...",Brighton Polytechnic,"[{""degree"":""2:2"",""end_year"":""1991"",""field"":""El...",https://media.licdn.com/dms/image/C4D03AQHLj-Z...,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-05-29,giovanna-panarella-99a0a4167,Giovanna Panarella,"Avellino, Campania, Italy",IT,EU,NaN,Freelance,Architetto (Freelance),500.0,...,"[{""profile_link"":""https://it.linkedin.com/in/e...",Università di Camerino,"[{""degree"":""“Corso di aggiornamento profession...",https://media-exp1.licdn.com/dms/image/C4D03AQ...,NaN,NaN,NaN,NaN,"[{""cause"":""Arts and Culture"",""duration"":""Jan 2...",NaN
4,2022-12-06,steve-latimer-3364327,Steve Latimer,"Ontario, Canada",CA,NaN,mid-range-computer-group-inc.,Mid-Range Computer Group Inc.,Senior Account Executive at Mid-Range Computer...,NaN,...,"[{""profile_link"":""https://ca.linkedin.com/in/d...",St. Michael's College School,"[{""degree"":"""",""end_year"":""1978"",""field"":"""",""me...",NaN,NaN,"[{""meta"":""Issued Jan 2022 See credential"",""sub...","Blake Reeves “If I was a customer, I would wan...",1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-03-29,lore-geduhn-58978171,Lore Geduhn,"Hagen, North Rhine-Westphalia, Germany",DE,EU,NaN,Awwads Lernzirkel,Bürokraft bei Awwads Lernzirkel,1.0,...,NaN,NaN,NaN,https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,NaN,NaN,NaN,NaN,NaN,NaN
996,2022-01-23,aaron-rodzinak-6005403,Aaron Rodzinak,"Boulder, Colorado, United States",US,NaN,ball-aerospace,Ball Aerospace,"Principal Spacecraft Systems Engineer, Ball Ae...",397.0,...,"[{""profile_link"":""https://www.linkedin.com/in/...",Embry-Riddle Aeronautical University,"[{""degree"":""BSAE"",""end_year"":""1994"",""field"":""A...",https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,NaN,NaN,NaN,NaN,NaN,NaN
997,2021-04-30,carmen-nemoianu-a2130869,carmen nemoianu,Romania,RO,AS,NaN,NaN,medic,29.0,...,NaN,facultatea de medicina Timisoara Education fac...,"[{""degree"":""medic primar -doctor in medicina"",...",https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,NaN,NaN,NaN,NaN,NaN,NaN
998,2022-02-09,carlos-soriano-51022049,Carlos Soriano,Greater Houston,US,NaN,NaN,NaN,Seeking,457.0,...,"[{""profile_link"":""https://www.linkedin.com/in/...",Lee University,"[{""degree"":""Bachelor of Business Administratio...",https://media-exp1.licdn.com/dms/image/C4E03AQ...,"[{""subtitle"":""-"",""title"":""Spanish""},{""subtitle...",NaN,NaN,NaN,NaN,NaN


In [12]:
profiles = profiles.dropna(subset=['position', 'about'])
profiles.isnull().sum()

timestamp                       0
id                              0
name                            0
city                           51
country_code                    2
region                        346
current_company:company_id    171
current_company:name           50
position                        0
following                     327
about                           0
posts                         260
groups                        434
current_company                 0
experience                      8
url                             0
people_also_viewed             54
educations_details             50
education                      22
avatar                        105
languages                     298
certifications                347
recommendations               356
recommendations_count         356
volunteer_experience          400
сourses                       491
dtype: int64

In [13]:
# Translate Teks to English

!pip install deep_translator
!pip install tqdm
!pip install langdetect

from deep_translator import GoogleTranslator
from tqdm import tqdm
from langdetect import detect

In [14]:
tqdm.pandas()
def translate_to_english(translate):
  if isinstance(translate, str):  # Check if the input is a string
    try:
        detected_lang = detect(translate)
        translated_text = GoogleTranslator(source=detected_lang, target='en').translate(translate)
        return translated_text
    except Exception as e:
        print(f"Error translating text: {translate}. Error: {e}")
        return translate
  return translate
clean_profiles = profiles.copy()
clean_profiles['position'] = clean_profiles['position'].progress_apply(translate_to_english)
clean_profiles['about'] = clean_profiles['about'].progress_apply(translate_to_english)
clean_profiles

  6%|▌         | 30/497 [00:06<00:49,  9.43it/s]

Error translating text: --. Error: No features in text.


 21%|██        | 105/497 [00:13<00:25, 15.64it/s]

Error translating text: --. Error: No features in text.


 48%|████▊     | 239/497 [00:26<00:24, 10.50it/s]

...

Error translating text: --. Error: No features in text.


 87%|████████▋ | 433/497 [00:48<00:02, 30.37it/s]

Error translating text: --. Error: No features in text.


100%|██████████| 497/497 [00:53<00:00,  9.24it/s]


,timestamp,id,name,city,country_code,region,current_company:company_id,current_company:name,position,following,...,people_also_viewed,educations_details,education,avatar,languages,certifications,recommendations,recommendations_count,volunteer_experience,сourses
1,2022-12-17,margot-bon-51a04624,Margot Bon,"The Randstad, Netherlands",NL,EU,gemeente-utrecht,Gemeente Utrecht,Communicatieadvisor Corporate & Strategy Gemee...,NaN,...,"[{""profile_link"":""https://nl.linkedin.com/in/j...",NaN,"[{""degree"":""Scrum en Agile werken"",""end_year"":...",https://static.licdn.com/sc/h/244xhbkr7g40x6bs...,"[{""subtitle"":""Professional working proficiency...","[{""meta"":""Issued Jun 2013"",""subtitle"":""Van der...",Menno H. Poort “Ik werk al jaren prettig met M...,2.0,"[{""cause"":"""",""duration"":""Sep 2010 Jul 2020 9 y...",NaN
2,2023-05-17,mike-dean-8509a193,Mike Dean,"England, United Kingdom",UK,NaN,network-rail,Network Rail,Network Data Manager at Network Rail,NaN,...,"[{""profile_link"":""https://uk.linkedin.com/in/g...",Brighton Polytechnic,"[{""degree"":""2:2"",""end_year"":""1991"",""field"":""El...",https://media.licdn.com/dms/image/C4D03AQHLj-Z...,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-03-23,gerardludovicwan,Gerard Ludovic Wan,Canada,CA,NaN,NaN,NaN,Aspiring Junior Engineer | McGill University G...,138.0,...,"[{""profile_link"":""https://sg.linkedin.com/in/s...",McGill University,"[{""degree"":""Bachelor of Engineering (B.Eng.)"",...",NaN,"[{""subtitle"":""Full professional proficiency"",""...","[{""meta"":""Issued Apr 2012"",""subtitle"":""ASP Con...",NaN,NaN,"[{""cause"":""Science and Technology"",""duration"":...",NaN
8,2021-07-14,kyle-huddle-a67a21b8,Kyle Huddle,"Allentown, Pennsylvania, United States",US,NaN,clear-channel-airports,Clear Channel Airport,Airport advertising creates significant brand ...,219.0,...,NaN,Lehigh Carbon Community College,"[{""degree"":"""",""end_year"":""2005"",""field"":"""",""me...",https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,NaN,NaN,NaN,NaN,NaN,NaN
10,2023-01-13,sarah-mcqueen-3bb78218,Sarah McQueen,"Carmel, Indiana, USA",US,NaN,NaN,"Therapets of Indiana,Inc.","Therapy dog handler, at Therapets of Indiana,Inc.",NaN,...,"[{""profile_link"":""https://www.linkedin.com/in/...",Butler University,"[{""degree"":""Bachelors"",""end_year"":""1990"",""fiel...",https://media.licdn.com/dms/image/C4D03AQEGNKX...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,2023-06-27,nagylagouveia,Nágyla Gouveia,Greater Fortaleza,BR,NaN,emilio-ribas,Emilio Ribas,Marketing | Endomarketing | Social Media,NaN,...,"[{""profile_link"":""https://br.linkedin.com/in/i...",Universidade 7 de Setembro - UNI7,"[{""degree"":""Bacharelado"",""end_year"":""2018"",""fi...",https://media.licdn.com/dms/image/C4D03AQGt2TU...,"[{""subtitle"":""Professional working proficiency...","[{""meta"":""Issued Sep 2018"",""subtitle"":""Google""...",Emanuelle Sena “Nágyla é uma ótima colega de t...,2.0,NaN,NaN
991,2022-12-12,haris-nadeem,Haris Nadeem,"Kent, Washington, United States",US,NaN,jll,JLL,Operating Engineer at JLL/Amazon,NaN,...,"[{""profile_link"":""https://www.linkedin.com/in/...",University of South Florida,"[{""degree"":""Master of Engineering - MEng"",""end...",https://media-exp1.licdn.com/dms/image/C5603AQ...,"[{""subtitle"":""Native or bilingual proficiency""...","[{""meta"":""Issued Nov 2021 See credential"",""sub...",Muhammad Rovaid khan “Haris has strong communi...,1.0,"[{""cause"":""Education"",""duration"":"""",""duration_...",NaN
993,2023-01-16,mohamed-ibrahim-roushdy-cia-b95b067,"Mohamed Ibrahim Roushdy , CIA",Egypt,EG,AF,abu-qir-petrolume,Abu Qir Petrolum,Certified Internal Auditor(CIA) Internal Audit...,NaN,...,"[{""profile_link"":""https://tr.linkedin.com/in/s...",Institute of Internal Auditors IIA,"[{""degree"":""Certified Internal Auditor - CIA"",...",NaN,"[{""subtitle"":""-"",""title"":""English""},{""subtitle...","[{""meta"":""Issued Jul 2013"",""subtitle"":

In [15]:
# Preprocess text function
def preprocess_text1(text1):
    if isinstance(text1, str):
        # Lowercase
        text1 = text1.lower()
        # Remove symbols
        text1 = re.sub(r'[^A-Za-z\s]', '', text1)
        # Remove stopwords
        stop1 = set(stopwords.words('english'))
        text1 = ' '.join([word for word in text1.split() if word not in stop1])
        return text1
    return ''

# Apply preprocessing
clean_profiles['position'] = clean_profiles['position'].apply(preprocess_text1)
clean_profiles['about'] = clean_profiles['about'].apply(preprocess_text1)
clean_profiles['combined'] = clean_profiles['position'] + ' ' + clean_profiles['about']
clean_profiles

,timestamp,id,name,city,country_code,region,current_company:company_id,current_company:name,position,following,...,educations_details,education,avatar,languages,certifications,recommendations,recommendations_count,volunteer_experience,сourses,combined
1,2022-12-17,margot-bon-51a04624,Margot Bon,"The Randstad, Netherlands",NL,EU,gemeente-utrecht,Gemeente Utrecht,communicatieadvisor corporate strategy gemeent...,NaN,...,NaN,"[{""degree"":""Scrum en Agile werken"",""end_year"":...",https://static.licdn.com/sc/h/244xhbkr7g40x6bs...,"[{""subtitle"":""Professional working proficiency...","[{""meta"":""Issued Jun 2013"",""subtitle"":""Van der...",Menno H. Poort “Ik werk al jaren prettig met M...,2.0,"[{""cause"":"""",""duration"":""Sep 2010 Jul 2020 9 y...",NaN,communicatieadvisor corporate strategy gemeent...
2,2023-05-17,mike-dean-8509a193,Mike Dean,"England, United Kingdom",UK,NaN,network-rail,Network Rail,network data manager network rail,NaN,...,Brighton Polytechnic,"[{""degree"":""2:2"",""end_year"":""1991"",""field"":""El...",https://media.licdn.com/dms/image/C4D03AQHLj-Z...,NaN,NaN,NaN,NaN,NaN,NaN,network data manager network rail experienced ...
6,2022-03-23,gerardludovicwan,Gerard Ludovic Wan,Canada,CA,NaN,NaN,NaN,aspiring junior engineer mcgill university gra...,138.0,...,McGill University,"[{""degree"":""Bachelor of Engineering (B.Eng.)"",...",NaN,"[{""subtitle"":""Full professional proficiency"",""...","[{""meta"":""Issued Apr 2012"",""subtitle"":""ASP Con...",NaN,NaN,"[{""cause"":""Science and Technology"",""duration"":...",NaN,aspiring junior engineer mcgill university gra...
8,2021-07-14,kyle-huddle-a67a21b8,Kyle Huddle,"Allentown, Pennsylvania, United States",US,NaN,clear-channel-airports,Clear Channel Airport,airport advertising creates significant brand ...,219.0,...,Lehigh Carbon Community College,"[{""degree"":"""",""end_year"":""2005"",""field"":"""",""me...",https://static-exp1.licdn.com/sc/h/244xhbkr7g4...,NaN,NaN,NaN,NaN,NaN,NaN,airport advertising creates significant brand ...
10,2023-01-13,sarah-mcqueen-3bb78218,Sarah McQueen,"Carmel, Indiana, USA",US,NaN,NaN,"Therapets of Indiana,Inc.",therapy dog handler therapets indianainc,NaN,...,Butler University,"[{""degree"":""Bachelors"",""end_year"":""1990"",""fiel...",https://media.licdn.com/dms/image/C4D03AQEGNKX...,NaN,NaN,NaN,NaN,NaN,NaN,therapy dog handler therapets indianainc worke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,2023-06-27,nagylagouveia,Nágyla Gouveia,Greater Fortaleza,BR,NaN,emilio-ribas,Emilio Ribas,marketing endomarketing social media,NaN,...,Universidade 7 de Setembro - UNI7,"[{""degree"":""Bacharelado"",""end_year"":""2018"",""fi...",https://media.licdn.com/dms/image/C4D03AQGt2TU...,"[{""subtitle"":""Professional working proficiency...","[{""meta"":""Issued Sep 2018"",""subtitle"":""Google""...",Emanuelle Sena “Nágyla é uma ótima colega de t...,2.0,NaN,NaN,marketing endomarketing social media experienc...
991,2022-12-12,haris-nadeem,Haris Nadeem,"Kent, Washington, United States",US,NaN,jll,JLL,operating engineer jllamazon,NaN,...,University of South Florida,"[{""degree"":""Master of Engineering - MEng"",""end...",https://media-exp1.licdn.com/dms/image/C5603AQ...,"[{""subtitle"":""Native or bilingual proficiency""...","[{""meta"":""Issued Nov 2021 See credential"",""sub...",Muhammad Rovaid khan “Haris has strong communi...,1.0,"[{""cause"":""Education"",""duration"":"""",""duration_...",NaN,operating engineer jllamazon haris started mec...
993,2023-01-16,mohamed-ibrahim-roushdy-cia-b95b067,"Mohamed Ibrahim Roushdy , CIA",Egypt,EG,AF,abu-qir-petrolume,Abu Qir Petrolum,certified internal auditorcia internal audit i...,NaN,...,Institute of Internal Auditors IIA,"[{""degree"":""Certified Internal Auditor - CIA"",...",NaN,"[{""subtitle"":""-"",""title"":""English""},{""subtitle...","[{""meta"":""Issued Jul 2013"",""subtitle"":""IIA-The...",NaN,NaN,"[{""cause"":"""",""duration"":""May 2014 - Prese

In [16]:
clean_profiles = clean_profiles[['id', 'position', 'about', 'combined']]
clean_profiles.to_csv('clean_profiles.csv', index=False, sep=',')